In [1]:
%pip install SoccerNet
%pip install pytorchvideo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.0 MB/s eta 0:00:00


In [ ]:
import os
import zipfile
from SoccerNet.Downloader import SoccerNetDownloader as SNdl

# Set up the downloader
local_directory = "path/to/SoccerNet"
mySNdl = SNdl(LocalDirectory=local_directory)

# Download the data
mySNdl.downloadDataTask(task="mvfouls", split=["train", "valid", "test", "challenge"], password="s0cc3rn3t")

# Unzip the downloaded files
task_directory = os.path.join(local_directory, "mvfouls")
for split in ["train", "valid", "test", "challenge"]:
    zip_file = os.path.join(task_directory, f"{split}.zip")
    if os.path.exists(zip_file):
        # Create a new folder with the same name as the zip file
        extract_folder = os.path.join(task_directory, split)
        os.makedirs(extract_folder, exist_ok=True)

        # Extract the contents to the new folder
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)
        print(f"Extracted {split}.zip to {extract_folder}")
    else:
        print(f"{split}.zip not found")

# Optionally, remove the zip files after extraction
for split in ["train", "valid", "test", "challenge"]:
    zip_file = os.path.join(task_directory, f"{split}.zip")
    if os.path.exists(zip_file):
        os.remove(zip_file)
        print(f"Removed {split}.zip")

In [11]:
# Class name to label index

EVENT_DICTIONARY_action_class = {"Tackling":0,"Standing tackling":1,"High leg":2,"Holding":3,"Pushing":4,
                        "Elbowing":5, "Challenge":6, "Dive":7, "Dont know":8}

INVERSE_EVENT_DICTIONARY_action_class = {0:"Tackling", 1:"Standing tackling", 2:"High leg", 3:"Holding", 4:"Pushing",
                        5:"Elbowing", 6:"Challenge", 7:"Dive", 8:"Dont know"}


EVENT_DICTIONARY_offence_severity_class = {"No offence":0,"Offence + No card":1,"Offence + Yellow card":2,"Offence + Red card":3}

INVERSE_EVENT_DICTIONARY_offence_severity_class = {0:"No offence", 1:"Offence + No card", 2:"Offence + Yellow card", 3:"Offence + Red card"}


EVENT_DICTIONARY_offence_class = {"Offence":0,"Between":1,"No Offence":2, "No offence":2}

INVERSE_EVENT_DICTIONARY_offence_class = {0:"Offence", 1:"Between", 2:"No Offence"}


EVENT_DICTIONARY_severity_class = {"1.0":0,"2.0":1,"3.0":2,"4.0":3,"5.0":4}

INVERSE_EVENT_DICTIONARY_severity_class = {0:"No card", 1:"Borderline No/Yellow", 2:"Yellow card", 3:"Borderline Yellow/Red", 4:"Red card"}


EVENT_DICTIONARY_bodypart_class = {"Upper body":0,"Under body":1}

INVERSE_EVENT_DICTIONARY_bodypart_class = {0:"Upper body", 1:"Under body"}



EVENT_DICTIONARY = {'action_class':EVENT_DICTIONARY_action_class, 'offence_class': EVENT_DICTIONARY_offence_class,
            'severity_class': EVENT_DICTIONARY_severity_class, 'bodypart_class': EVENT_DICTIONARY_bodypart_class, 'offence_severity_class': EVENT_DICTIONARY_offence_severity_class}
INVERSE_EVENT_DICTIONARY = {'action_class':INVERSE_EVENT_DICTIONARY_action_class, 'offence_class': INVERSE_EVENT_DICTIONARY_offence_class,
            'severity_class': INVERSE_EVENT_DICTIONARY_severity_class, 'bodypart_class': INVERSE_EVENT_DICTIONARY_bodypart_class, 'offence_severity_class': INVERSE_EVENT_DICTIONARY_offence_severity_class}

In [51]:
import os
import torch
import json
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#DATA_PATH = 'path/to/SoccerNet/mvfouls'

# Set the desired frame count
DESIRED_FRAME_COUNT = 126

# Load the EVENT_DICTIONARY for mapping annotation labels
EVENT_DICTIONARY = {
    'action_class': {"Tackling": 0, "Standing tackling": 1, "High leg": 2, "Holding": 3, "Pushing": 4,
                     "Elbowing": 5, "Challenge": 6, "Dive": 7, "Dont know": 8},
    'offence_class': {"Offence": 0, "Between": 1, "No Offence": 2, "No offence": 2},
    'severity_class': {"1.0": 0, "2.0": 1, "3.0": 2, "4.0": 3, "5.0": 4},
    'bodypart_class': {"Upper body": 0, "Under body": 1},
    'offence_severity_class': {"No offence": 0, "Offence + No card": 1, "Offence + Yellow card": 2, "Offence + Red card": 3}
}

# Transformation for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def load_filtered_clips_and_labels(DATA_PATH,split, max_samples):
    clips, labels_action, labels_offence, labels_severity, labels_bodypart, labels_offence_severity = [], [], [], [], [], []

    annotations_path = os.path.join(DATA_PATH, split, "annotations.json")
    print(f"Loading annotations from: {annotations_path}")

    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    print(f"Total actions found in annotations: {len(annotations['Actions'])}")

    offence_count, no_offence_count, skipped_actions = 0, 0, 0
    max_samples = max_samples  # Maximum samples for each class

    for action_index, (action_key, action_data) in enumerate(annotations['Actions'].items()):
        offence_class = action_data['Offence']

        # Filter only 50 samples for each class
        if (offence_class == "Offence" and offence_count >= max_samples) or \
           (offence_class in ["No offence", "No Offence"] and no_offence_count >= max_samples):
            continue

        action_class = action_data['Action class']
        severity_class = action_data['Severity'] if action_data['Severity'].replace('.', '').isdigit() else '1.0'
        bodypart_class = action_data.get('Bodypart', 'Upper body')
        offence_severity = f"{offence_class} + {EVENT_DICTIONARY['severity_class'].get(severity_class, 'No card')}"

        action_label = EVENT_DICTIONARY['action_class'].get(action_class)
        offence_label = EVENT_DICTIONARY['offence_class'].get(offence_class)
        severity_label = EVENT_DICTIONARY['severity_class'].get(severity_class)
        bodypart_label = EVENT_DICTIONARY['bodypart_class'].get(bodypart_class)
        offence_severity_label = EVENT_DICTIONARY['offence_severity_class'].get(offence_severity, 0)

        if None in [action_label, offence_label, severity_label, bodypart_label, offence_severity_label]:
            skipped_actions += 1
            continue

        action_folder = os.path.join(DATA_PATH, split, f"action_{action_key}")

        if not os.path.exists(action_folder):
            skipped_actions += 1
            continue

        action_clips = []
        for clip_idx in range(2):
            clip_path = os.path.join(action_folder, f"clip_{clip_idx}.mp4")
            if not os.path.exists(clip_path):
                continue

            cap = cv2.VideoCapture(clip_path)
            frames = []
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)  # Convert NumPy array to PIL Image
                frame = transform(frame)       # Apply the updated transform
                frames.append(frame)
            cap.release()

            # Sample or pad frames to ensure uniform frame count
            if len(frames) > DESIRED_FRAME_COUNT:
                indices = np.linspace(0, len(frames) - 1, DESIRED_FRAME_COUNT).astype(int)
                frames = [frames[i] for i in indices]
            elif len(frames) < DESIRED_FRAME_COUNT:
                frames += [frames[-1]] * (DESIRED_FRAME_COUNT - len(frames))

            video_tensor = torch.stack(frames, dim=0)
            action_clips.append(video_tensor)


            # Print tensor shape and size for debugging
            print(f"Action {action_key}, Clip {clip_idx} tensor shape: {video_tensor.shape}")
            tensor_size = video_tensor.element_size() * video_tensor.nelement() / (1024**2)
            print(f"Tensor size: {tensor_size:.2f} MB")

        if action_clips:
            clips.append(action_clips)
            labels_action.append(action_label)
            labels_offence.append(offence_label)
            labels_severity.append(severity_label)
            labels_bodypart.append(bodypart_label)
            labels_offence_severity.append(offence_severity_label)

            if offence_class == "Offence":
                offence_count += 1
            else:
                no_offence_count += 1

            print(f"Added action {action_key} with {len(action_clips)} clips to dataset.")

        # Stop if we have 50 samples for each class
        if offence_count >= max_samples and no_offence_count >= max_samples:
            break

    print("\nSummary:")
    print(f"Total actions loaded: {len(clips)}")
    print(f"Total actions skipped: {skipped_actions}")
    if clips:
        print(f"Example clip shape: {clips[0][0].shape}")
        print(f"First action label: {labels_action[0]}")

    return clips, labels_action, labels_offence, labels_severity, labels_bodypart, labels_offence_severity


# # Load datasets for each split
# datasets = {}
# for split in ['train', 'valid', 'test']:
#     clips, labels_action, labels_offence, labels_severity, labels_bodypart, labels_offence_severity = load_filtered_clips_and_labels(DATA_PATH,split)
#     datasets[split] = {
#         'clips': clips,
#         'labels': {
#             'action': labels_action,
#             'offence': labels_offence,
#             'severity': labels_severity,
#             'bodypart': labels_bodypart,
#             'offence_severity': labels_offence_severity
#         }
#     }

# # Display dataset info for verification
# for split in ['train', 'valid', 'test']:
#     print(f"{split.capitalize()} set:")
#     print("Number of samples:", len(datasets[split]['clips']))
#     print("Example label structure:", datasets[split]['labels']['action'][0] if datasets[split]['labels']['action'] else "No labels")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from pytorchvideo.models.hub import mvit_base_16x4
from torchvision.transforms import Compose, Normalize

class VideoDataset(Dataset):
    def __init__(self, clips, labels, transform=None):
        self.clips = clips
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.clips)

    def __getitem__(self, idx):
        video = self.clips[idx]  # [T, C, H, W]
        label = self.labels[idx]  # Scalar label

        # Apply transformations to all frames
        if self.transform:
            video = torch.stack([self.transform(frame) for frame in video], dim=0)  # [T, C, H, W]

        # Permute dimensions to [C, T, H, W] (required for Conv3D input)
        video = video.permute(1, 0, 2, 3)

        return video, label


# Define the model
class MViTModel(nn.Module):
    def __init__(self, num_classes):
        super(MViTModel, self).__init__()
        self.mvit = mvit_base_16x4(pretrained=False)
        self.mvit.head.proj = nn.Linear(self.mvit.head.proj.in_features, num_classes)  # Adjust final layer

    def forward(self, x):
        return self.mvit(x)

# Dataset and DataLoader
def create_dataloaders(clips, labels, batch_size, transform, split_ratios=(0.7, 0.15, 0.15)):
    # Split data into train, val, and test
    data_size = len(clips)
    train_size = int(split_ratios[0] * data_size)
    val_size = int(split_ratios[1] * data_size)
    test_size = data_size - train_size - val_size

    train_clips, train_labels = clips[:train_size], labels[:train_size]
    val_clips, val_labels = clips[train_size:train_size + val_size], labels[train_size:train_size + val_size]
    test_clips, test_labels = clips[train_size + val_size:], labels[train_size + val_size:]

    # Create datasets
    train_dataset = VideoDataset(train_clips, train_labels, transform=transform)
    val_dataset = VideoDataset(val_clips, val_labels, transform=transform)
    test_dataset = VideoDataset(test_clips, test_labels, transform=transform)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Training function
def train_model(model, train_loader, val_loader, num_epochs, device, lr=1e-4):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_correct = 0.0, 0
        total_samples = 0

        for videos, labels in train_loader:
            print(f"Video batch shape: {videos.shape}")  # Should be [batch_size, channels, depth, height, width]
            print(f"Labels shape: {labels.shape}")  # Should be [batch_size]
            videos, labels = videos.to(device), labels.to(device)
            outputs = model(videos)

            optimizer.zero_grad()
            #outputs = model(videos)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * videos.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
            total_samples += videos.size(0)

        train_loss /= total_samples
        train_accuracy = train_correct / total_samples

        # Validate
        model.eval()
        val_loss, val_correct = 0.0, 0
        total_samples = 0

        with torch.no_grad():
            for videos, labels in val_loader:
                videos, labels = videos.to(device), labels.to(device)

                outputs = model(videos)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * videos.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                total_samples += videos.size(0)

        val_loss /= total_samples
        val_accuracy = val_correct / total_samples

        print(f"Epoch {epoch + 1}/{num_epochs}:")
        print(f"    Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"    Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    test_correct = 0
    total_samples = 0

    with torch.no_grad():
        for videos, labels in test_loader:
            videos, labels = videos.to(device), labels.to(device)

            outputs = model(videos)
            _, predicted = torch.max(outputs, 1)
            test_correct += (predicted == labels).sum().item()
            total_samples += videos.size(0)

    test_accuracy = test_correct / total_samples
    print(f"Test Accuracy: {test_accuracy:.4f}")

# Main script
if __name__ == "__main__":
    # Example parameters
    NUM_CLASSES = 9  # Adjust based on your dataset
    BATCH_SIZE = 2
    NUM_EPOCHS = 10
    DATA_PATH = 'mvfouls'
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load your processed clips and labels
    # Replace with your actual data
    # Example: clips, labels = load_filtered_clips_and_labels(DATA_PATH, 'train', max_samples=50)
    clips, labels_action, _, _, _, _ = load_filtered_clips_and_labels(DATA_PATH, 'train', max_samples=5)

    labels = labels_action

    # Transformations
    transform = Compose([
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for pretrained models
    ])

    # Create DataLoaders
    train_loader, val_loader, test_loader = create_dataloaders(clips, labels, BATCH_SIZE, transform)

    # Initialize and train the model
    model = MViTModel(num_classes=NUM_CLASSES)
    train_model(model, train_loader, val_loader, NUM_EPOCHS, DEVICE)

    # Evaluate the model
    evaluate_model(model, test_loader, DEVICE)


Loading annotations from: mvfouls\train\annotations.json
Total actions found in annotations: 2916
Action 0, Clip 0 tensor shape: torch.Size([126, 3, 224, 224])
Tensor size: 72.35 MB
Action 0, Clip 1 tensor shape: torch.Size([126, 3, 224, 224])
Tensor size: 72.35 MB
Added action 0 with 2 clips to dataset.
Action 1, Clip 0 tensor shape: torch.Size([126, 3, 224, 224])
Tensor size: 72.35 MB
Action 1, Clip 1 tensor shape: torch.Size([126, 3, 224, 224])
Tensor size: 72.35 MB
Added action 1 with 2 clips to dataset.
Action 2, Clip 0 tensor shape: torch.Size([126, 3, 224, 224])
Tensor size: 72.35 MB
Action 2, Clip 1 tensor shape: torch.Size([126, 3, 224, 224])
Tensor size: 72.35 MB
Added action 2 with 2 clips to dataset.
Action 3, Clip 0 tensor shape: torch.Size([126, 3, 224, 224])
Tensor size: 72.35 MB
Action 3, Clip 1 tensor shape: torch.Size([126, 3, 224, 224])
Tensor size: 72.35 MB
Added action 3 with 2 clips to dataset.
Action 4, Clip 0 tensor shape: torch.Size([126, 3, 224, 224])
Tensor s

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 5 is not equal to len(dims) = 4